# Algoritmos Genéticos

In [49]:
import random
import numpy as np

### Fichas

In [50]:
fichas = {
    "a": 10,
    "b": 3,
    "c": 7,
    "d": 1,
    "e": 2,
    "f": 8,
    "g": 4,
    "h": 10,
    "i": 9,
    "j": 3
}

## Clase Individuos

- **generate_random_array:** generación de cada individuo con **a** y **j** en la primera y última posición, generándose de forma aleatoria los otros 6 valores para tener cada individuo con 8 cromosomas
- **fitness_function:** aplicación de la **función de calidad** a cada cromosoma del individuo y se aplica la función inversa a cada uno (Se realiza una multiplicación con los adyacentes)
- **function_adition:** suma de la función y la función inversa de cada cromosoma para obtener el total correspondiente al individuo

In [51]:
class Individuos():
    def __init__(self, fichas):
        self.fichas = fichas
        self.individuo = []
        self.function = []
        self.inverse_function = []
        self.total_function = 0
        self.total_inverse_function = 0
        self.probabilidad = 0
        self.probabilidad_inversa = 0
        self.probabilidad_acumulada = []

    def generate_random_array(self):
        random_array = [random.randint(2, 9) for _ in range(6)]
        self.individuo.append(fichas[list(fichas.keys())[0]])
        for i in range(6):
            self.individuo.append(fichas[random.choice(list(fichas.keys()))])
        self.individuo.append(fichas[list(fichas.keys())[-1]])
    
    def fitness_function(self):
        for i in range(len(self.individuo)):
            if i == 0:
                self.function.append(self.individuo[i] * self.individuo[i + 1])
            elif i == len(self.individuo) - 1:
                self.function.append(self.individuo[i] * self.individuo[i - 1])
            else:
                self.function.append(self.individuo[i] * self.individuo[i - 1] * self.individuo[i + 1])
            self.inverse_function.append(1 / self.function[i])
        
        for value in self.function:
            self.inverse_function.append(1 / value)
    
    def function_addition(self):
        for i in range(len(self.function)):
            self.total_function += self.function[i]
        self.total_inverse_function = 1/self.total_function
            

### Implementación

In [52]:
individuo = Individuos(fichas)
individuo.generate_random_array()
individuo.fitness_function()
individuo.function_addition()

print("Individuo:", individuo.individuo)
print("Fitness Function:", individuo.function)
print("Inverse Function:", individuo.inverse_function)
print("Total Function:", individuo.total_function)
print("Total Inverse Function:", individuo.total_inverse_function)

Individuo: [10, 10, 9, 10, 3, 3, 1, 3]
Fitness Function: [100, 900, 900, 270, 90, 9, 9, 3]
Inverse Function: [0.01, 0.0011111111111111111, 0.0011111111111111111, 0.003703703703703704, 0.011111111111111112, 0.1111111111111111, 0.1111111111111111, 0.3333333333333333, 0.01, 0.0011111111111111111, 0.0011111111111111111, 0.003703703703703704, 0.011111111111111112, 0.1111111111111111, 0.1111111111111111, 0.3333333333333333]
Total Function: 2281
Total Inverse Function: 0.00043840420868040335


## Creación de población y probabilidades

- **creacion_poblacion:** se genera una población (array) con la cantidad de individuos que se asignen
- **probabilidades:** se calcula la probabilidad (total) de cada individuo, utilizando (1/f)/sum(f)

In [53]:
def creacion_poblacion(fichas, cantidad):
    poblacion = []
    for _ in range(cantidad):
        individuo = Individuos(fichas)
        individuo.generate_random_array()
        individuo.fitness_function()
        individuo.function_addition()
        poblacion.append(individuo)
    return poblacion

def probabilidades(poblacion):
    for individuo in poblacion:
        individuo.probabilidad = individuo.total_inverse_function / sum(ind.total_inverse_function for ind in poblacion)

def probabilidades_acum(poblacion):
    acumulada = 0
    for individuo in poblacion:
        acumulada += individuo.probabilidad
        individuo.probabilidad_acumulada.append(acumulada)

# def probabilidades_inversa(poblacion):
#     for individuo in poblacion:
#         individuo.probabilidad_inversa = individuo.total_inverse_function / sum(ind.total_inverse_function for ind in poblacion)

### Implementación

In [54]:
poblacion = creacion_poblacion(fichas, 5)
probabilidades(poblacion)
probabilidades_acum(poblacion)

print("Probabilidades Acumuladas:")
for ind in poblacion:
    print(f"Individuo: {ind.individuo}, Funcion: {ind.total_function}, Probabilidad: {ind.probabilidad}, Probabilidad Acumulada: {ind.probabilidad_acumulada}")

print("Total Probabilidad:", sum(ind.probabilidad for ind in poblacion))

Probabilidades Acumuladas:
Individuo: [10, 4, 10, 3, 3, 4, 9, 3], Funcion: 929, Probabilidad: 0.2272175330032746, Probabilidad Acumulada: [0.2272175330032746]
Individuo: [10, 10, 8, 10, 3, 4, 1, 3], Funcion: 2087, Probabilidad: 0.10114283093437573, Probabilidad Acumulada: [0.32836036393765033]
Individuo: [10, 8, 8, 10, 10, 8, 3, 3], Funcion: 3281, Probabilidad: 0.06433559529413048, Probabilidad Acumulada: [0.3926959592317808]
Individuo: [10, 4, 3, 10, 1, 10, 7, 3], Funcion: 711, Probabilidad: 0.29688479347403957, Probabilidad Acumulada: [0.6895807527058204]
Individuo: [10, 2, 3, 3, 10, 7, 3, 3], Funcion: 680, Probabilidad: 0.3104192472941796, Probabilidad Acumulada: [1.0]
Total Probabilidad: 1.0


## Selección

- El individuo con mayor función de calidad obtiene mayor probabilidad de ser seleccionado en el método de la ruleta (Se utiliza 1/f debido a que en el problema se está intentando minimizar)
- Se deben seleccionar 2 individuos para realizar mutación o cruce

In [55]:
def wheel_selection(poblacion):
    random_value = random.random()
    print("Random Value:", random_value)
    for ind in poblacion:
        if random_value <= ind.probabilidad_acumulada[-1]:
            return ind
    return poblacion[-1]

In [56]:
flag = True
while flag:
    individuo1 = wheel_selection(poblacion)
    individuo2 = wheel_selection(poblacion)
    if individuo1 == individuo2:
        print("Selected the same individual, re-selecting...")
        continue
    else:
        flag = False

print("Final Selected Individual 1:", individuo1.individuo, "with Probability:", individuo1.probabilidad)
print("Final Selected Individual 2:", individuo2.individuo, "with Probability:", individuo2.probabilidad)

Random Value: 0.08175347403316469
Random Value: 0.09582716783497602
Selected the same individual, re-selecting...
Random Value: 0.7191320505567657
Random Value: 0.7504410338392824
Selected the same individual, re-selecting...
Random Value: 0.12813651660086522
Random Value: 0.04633789070947836
Selected the same individual, re-selecting...
Random Value: 0.03567507325776187
Random Value: 0.8584878350374844
Final Selected Individual 1: [10, 4, 10, 3, 3, 4, 9, 3] with Probability: 0.2272175330032746
Final Selected Individual 2: [10, 2, 3, 3, 10, 7, 3, 3] with Probability: 0.3104192472941796


## Cruces

In [57]:
def crossover(ind1, ind2):
    crossover_point = random.randint(1, len(ind1.individuo) - 2)
    child1 = ind1.individuo[:crossover_point] + ind2.individuo[crossover_point:]
    child2 = ind2.individuo[:crossover_point] + ind1.individuo[crossover_point:]
    return child1, child2

child1, child2 = crossover(individuo1, individuo2)
print("Child 1:", child1)
print("Child 2:", child2)

Child 1: [10, 4, 10, 3, 10, 7, 3, 3]
Child 2: [10, 2, 3, 3, 3, 4, 9, 3]


## Mutaciones

In [58]:
def mutation(individuo, mutation_rate=0.1):
    for i in range(len(individuo.individuo)):
        if random.random() < mutation_rate:
            new_value = random.choice(list(fichas.values()))
            individuo.individuo[i] = new_value
    individuo.fitness_function()
    individuo.function_addition()
    print("Mutated Individual:", individuo.individuo, "with Total Function:", individuo.total_function)